# Exploratory Data Analysis - Sales Analytics Platform

Comprehensive data exploration notebook for the Sales Analytics Platform.

This notebook provides:
- Data loading and validation
- Summary statistics and data quality checks
- Sales trends over time
- Revenue analysis by region, channel, and product
- Customer segmentation insights
- Correlation and relationship analysis
- Professional interactive visualizations

Designed for business analysts, data scientists, and sales leaders.

## 1. Import Required Libraries

In [ ]:
# Import data analysis libraries
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Visualization libraries
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set seaborn style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully")

## 2. Load Sales Data

In [ ]:
# Load sales data
data_path = Path('../data/sales_data.csv')

if data_path.exists():
    df = pd.read_csv(data_path, parse_dates=['date'])
    print(f"✅ Data loaded successfully!")
    print(f"📊 Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"📅 Date range: {df['date'].min()} to {df['date'].max()}")
else:
    print("❌ Data file not found. Please run: python data/generate_sample_data.py")
    df = pd.DataFrame()

# Display first few rows
df.head()

## 3. Data Quality & Summary Statistics

In [ ]:
# Dataset info
print("=" * 60)
print("DATASET INFORMATION")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
print(df.describe())

print("\n" + "=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)
print(f"Missing values:\n{df.isnull().sum()}")
print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Key metrics
print("\n" + "=" * 60)
print("KEY BUSINESS METRICS")
print("=" * 60)
print(f"Total Revenue: ${df['revenue'].sum():,.2f}")
print(f"Total Orders: {df['order_id'].nunique():,}")
print(f"Avg Order Value: ${df['revenue'].mean():,.2f}")
print(f"Unique Customers: {df['customer_id'].nunique():,}")
print(f"Unique Products: {df['product_id'].nunique():,}")

## 4. Sales Trends Over Time

In [ ]:
# Daily sales trend
daily_sales = df.groupby(df['date'].dt.date).agg({
    'revenue': 'sum',
    'order_id': 'nunique',
    'quantity': 'sum'
}).reset_index()

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Daily Revenue Trend', 'Daily Order Volume'),
    vertical_spacing=0.12
)

fig.add_trace(
    go.Scatter(x=daily_sales['date'], y=daily_sales['revenue'], 
               name='Revenue', line=dict(color='#1f77b4', width=3)),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=daily_sales['date'], y=daily_sales['order_id'], 
           name='Orders', marker_color='#ff7f0e'),
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_yaxes(title_text="Orders", row=2, col=1)
fig.update_layout(height=600, showlegend=True, title_text="📈 Sales Performance Over Time")

fig.show()

## 5. Revenue Analysis by Dimensions

In [ ]:
# Revenue by Region, Channel, and Product
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Revenue by Region', 'Revenue by Channel', 'Top 10 Products'),
    specs=[[{'type': 'bar'}, {'type': 'pie'}, {'type': 'bar'}]]
)

# Region
region_revenue = df.groupby('region')['revenue'].sum().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=region_revenue.index, y=region_revenue.values, name='Region', marker_color='#2ecc71'),
    row=1, col=1
)

# Channel
channel_revenue = df.groupby('channel')['revenue'].sum()
fig.add_trace(
    go.Pie(labels=channel_revenue.index, values=channel_revenue.values, name='Channel'),
    row=1, col=2
)

# Top Products
product_revenue = df.groupby('product_id')['revenue'].sum().nlargest(10)
fig.add_trace(
    go.Bar(y=product_revenue.index, x=product_revenue.values, name='Product', 
           orientation='h', marker_color='#e74c3c'),
    row=1, col=3
)

fig.update_xaxes(title_text="Region", row=1, col=1)
fig.update_yaxes(title_text="Revenue ($)", row=1, col=1)
fig.update_xaxes(title_text="Revenue ($)", row=1, col=3)
fig.update_yaxes(title_text="Product ID", row=1, col=3)

fig.update_layout(height=500, showlegend=False, title_text="🗺️ Revenue Breakdown by Dimensions")
fig.show()

## 6. Customer Segmentation & Analysis

In [ ]:
# Customer Analysis
customer_metrics = df.groupby('customer_id').agg({
    'revenue': 'sum',
    'order_id': 'nunique',
    'quantity': 'sum',
    'date': ['min', 'max']
}).reset_index()

customer_metrics.columns = ['customer_id', 'total_revenue', 'order_count', 'total_quantity', 'first_order', 'last_order']
customer_metrics['avg_order_value'] = customer_metrics['total_revenue'] / customer_metrics['order_count']
customer_metrics['days_active'] = (customer_metrics['last_order'] - customer_metrics['first_order']).dt.days

# Customer segmentation by revenue
customer_metrics['segment'] = pd.qcut(customer_metrics['total_revenue'], q=4, labels=['Low', 'Medium', 'High', 'VIP'])

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Customer Segments (Revenue)', 'Avg Order Value by Segment')
)

# Segment distribution
segment_counts = customer_metrics['segment'].value_counts()
fig.add_trace(
    go.Pie(labels=segment_counts.index, values=segment_counts.values, name='Segments'),
    row=1, col=1
)

# AOV by segment
aov_by_segment = customer_metrics.groupby('segment')['avg_order_value'].mean().sort_values()
fig.add_trace(
    go.Bar(x=aov_by_segment.values, y=aov_by_segment.index, orientation='h', marker_color='#9b59b6'),
    row=1, col=2
)

fig.update_layout(height=400, showlegend=False, title_text="👥 Customer Segmentation Analysis")
fig.show()

print("📊 Customer Segment Summary:")
print(customer_metrics.groupby('segment').agg({
    'total_revenue': ['count', 'sum', 'mean'],
    'order_count': 'mean',
    'avg_order_value': 'mean'
}).round(2))

## 7. Correlation Analysis

In [ ]:
# Correlation heatmap
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=correlation_matrix.values.round(2),
    texttemplate='%{text}',
    textfont={"size": 10},
    colorbar=dict(title="Correlation")
))

fig.update_layout(
    title="Correlation Heatmap - Numeric Variables",
    height=600,
    xaxis={'side': 'bottom'},
    yaxis={'side': 'left'}
)

fig.show()

## 8. Key Insights & Recommendations

Based on the exploratory data analysis, here are the key insights:

### Sales Performance
- Analyze total revenue trends and identify peak and low periods
- Monitor daily sales volatility and seasonality patterns

### Geographic & Channel Insights
- Identify top-performing regions and channels
- Allocate resources to high-revenue segments

### Customer Behavior
- Understand customer segmentation (Low, Medium, High, VIP)
- Focus retention efforts on VIP and High-value customers
- Develop strategies to upgrade Medium and Low value customers

### Recommended Action Items
1. **Upselling**: Target VIP customers with premium products
2. **Cross-selling**: Bundle complementary products in top categories
3. **Retention**: Implement loyalty programs for high-value segments
4. **Expansion**: Invest marketing in high-performing regions

---

**Next Steps:**
- Proceed to ML Model Training Notebook for predictive analytics
- Use Custom Analysis Notebook for ad-hoc business questions
- Explore API Testing Notebook for system integration